# Regressão Múltipla

In [1]:
import numpy as np
import math
import time
from sklearn import linear_model

## 1. Regressão Múltipla com sklearn

Neste documento, a ideia é criar um modelo de regressão linear múltipla. Esse modelo será ajustado aos dados de notas de alunos de computação da UFCG em algumas disciplinas do primeiro período. A última coluna é a variável alvo representando o CRA final depois de concluir o curso. As outras colunas são algumas disciplinas do primeiro período. O pressuposto aqui é que as notas em disciplinas no primeiro período ajudam a explicar o CRA final dos alunos de computação.

Primeiramente, vou usar a biblioteca *sklearn* para criar o modelo. Depois de criado o modelo linear, a função *fit()* o ajusta aos dados (**X** contém todas as colunas de notas dos alunos e **Y** é o vetor com a coluna do cra).

In [2]:
grades = np.genfromtxt("sample_students_grades.csv", delimiter=",", skip_header=1)

X = grades[:,[0,1,2,3,4]]
Y = grades[:,5][:,np.newaxis]

startTime = time.time()
linear_reg = linear_model.LinearRegression() # this creates a linear regression object
linear_reg.fit(X,Y) #this function fits a linear model

endTime = time.time()
#print(linear_reg.score(X,Y)) #R² error
#print(linear_reg.coef_) # w1, w2, w3, ...
#print(linear_reg.intercept_) # w0
print("After {0} ms intercept = {1}, linear_coefficients = {2} error (R²) = {3}".format(str(1000*(endTime-startTime)), linear_reg.intercept_, linear_reg.coef_, linear_reg.score(X,Y)))

After 49.76296424865723 ms intercept = [1.73771151], linear_coefficients = [[0.10304143 0.0464367  0.16409834 0.38117843 0.02027816]] error (R²) = 0.423803240951211


## 2. Regressão Múltipla "do zero"

Agora, farei um modelo e o ajustarei aos dados sem utilizar a biblioteca. Ao final, compararei os coeficientes do novo modelo com os coeficientes do modelo criado anteriormente. Isso será o teste que indicará se o novo algoritmo está funcionando corretamente.



### 2.1 Mean Squared Error

$MSE(\hat{w})=\frac{1}{N}(y-\hat{\mathbf{w}}^T\mathbf{x})^T(y-\hat{\mathbf{w}}^T\mathbf{x})$

In [3]:
# Compute the Mean Squared Error
def compute_mse_vectorized(w,X,Y):
    res = Y - np.dot(X, w)
    totalError = np.dot(res.T,res)
    return totalError / float(len(Y))

### 2.2 Gradient


In [4]:

def step_gradient_vectorized(w_current,X,Y,learningRate):
    
    res = Y - np.dot(X, w_current) # (y−Hw)
   
    gradients = np.dot(X.T, res) # H.T (y−Hw)

    # α * -2HT (y−Hw): it didn't worked with this formula. 
    # So I replaced the (-2)  with the len(X) dividing
    rate = np.multiply(learningRate/len(X), gradients)    
    new_w = w_current + rate # w(t) − α * -2HT (y−Hw)
    
    return [new_w,gradients]

In [5]:
def gradient_descent_runner_vectorized(starting_w, X,Y, learning_rate, epsilon):
    w = starting_w
    gradients = np.array([np.inf])
    i = 0
    while (np.linalg.norm(gradients)>=epsilon):
        w,gradients = step_gradient_vectorized(w, X, Y, learning_rate)
        
        #print(np.linalg.norm(gradients))
        
        if i % 1000 == 0:
            print("MSE in the iteration {0} is equals to {1}".format(i,compute_mse_vectorized(w, X, Y)))
        i+= 1
    print("It converged with {0} iterations.".format(i))
    return w

#### Main

Como pode-se observar pela saída, o código converge com 212513 iterações quando os parâmetros são: **learning_rate = 0.0005** e **epsilon = 0.5**. Como pode ser visto no código acima (função *step_gradient_vectorized*), a taxa de aprendizado é dividida pelo tamanho de X. Não entendi muito bem o motivo, mas só convergiu dessa forma.

Depois de 2047,2838878631592 ms, w0 = 0,92369587, w1 = 0,111762742, w2 = 0,08368884, w3 = 0,16224365, w4 = 0,42251334  e w5 = 0,03029537, com erro de 0,41597456. 

Como é possível observar, esses valores se aproximam dos valores retornados pelo modelo criado com a biblioteca sklearn que foram: w0 =1.73771151, w1 = 0.10304143, w2 = 0.0464367, w3 = 0.16409834 w4 = 0.38117843 e w5 = 0.02027816, com erro de 0.423803240951211;


In [6]:
grades_incremented = np.c_[np.ones(len(grades)),grades]

X = grades_incremented[:,[0,1,2,3,4,5]] # grades without cra
Y = grades_incremented[:,6][:,np.newaxis] # cra
init_w = np.zeros((6,1)) # vector of coefficients

learning_rate = 0.0005
epsilon = 0.5

print("Starting gradient descent at w = {0}, error = {1}".format(init_w.T, compute_mse_vectorized(init_w, X,Y)))
print("Running...")
startTime = time.time()
w = gradient_descent_runner_vectorized(init_w, X,Y, learning_rate, epsilon)
endTime = time.time()
print("After {0} ms w = {1}, error = {2}".format(str(1000*(endTime-startTime)), w.T, compute_mse_vectorized(w,X,Y)))




Starting gradient descent at w = [[0. 0. 0. 0. 0. 0.]], error = [[54.47995386]]
Running...
MSE in the iteration 0 is equals to [[40.01097793]]
MSE in the iteration 1000 is equals to [[0.46211023]]
MSE in the iteration 2000 is equals to [[0.44714364]]
MSE in the iteration 3000 is equals to [[0.43977622]]
MSE in the iteration 4000 is equals to [[0.43584635]]
MSE in the iteration 5000 is equals to [[0.43363992]]
MSE in the iteration 6000 is equals to [[0.43235204]]
MSE in the iteration 7000 is equals to [[0.43156975]]
MSE in the iteration 8000 is equals to [[0.43107028]]
MSE in the iteration 9000 is equals to [[0.43073036]]
MSE in the iteration 10000 is equals to [[0.43048093]]
MSE in the iteration 11000 is equals to [[0.43028308]]
MSE in the iteration 12000 is equals to [[0.43011481]]
MSE in the iteration 13000 is equals to [[0.42996369]]
MSE in the iteration 14000 is equals to [[0.42982265]]
MSE in the iteration 15000 is equals to [[0.42968771]]
MSE in the iteration 16000 is equals to [

MSE in the iteration 165000 is equals to [[0.41779251]]
MSE in the iteration 166000 is equals to [[0.41774772]]
MSE in the iteration 167000 is equals to [[0.41770325]]
MSE in the iteration 168000 is equals to [[0.41765909]]
MSE in the iteration 169000 is equals to [[0.41761523]]
MSE in the iteration 170000 is equals to [[0.41757168]]
MSE in the iteration 171000 is equals to [[0.41752842]]
MSE in the iteration 172000 is equals to [[0.41748547]]
MSE in the iteration 173000 is equals to [[0.41744282]]
MSE in the iteration 174000 is equals to [[0.41740046]]
MSE in the iteration 175000 is equals to [[0.4173584]]
MSE in the iteration 176000 is equals to [[0.41731663]]
MSE in the iteration 177000 is equals to [[0.41727514]]
MSE in the iteration 178000 is equals to [[0.41723395]]
MSE in the iteration 179000 is equals to [[0.41719304]]
MSE in the iteration 180000 is equals to [[0.41715242]]
MSE in the iteration 181000 is equals to [[0.41711208]]
MSE in the iteration 182000 is equals to [[0.4170